In [14]:
import openai
import os
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain


# use reddis as a backend
from langchain.vectorstores.redis import Redis
redis_url = "redis://localhost:6379"

# read local .env file
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) 


# setup azure openai api
openai.api_type = "azure"
openai.api_base = os.getenv("AZURE_OAI_ENDPOINT")
openai.api_version = "2023-03-15-preview"
openai.api_key  = os.getenv('AZURE_OAI_KEY')

llm_name = "gpt-3.5-turbo"
deployment_name_gpt = os.getenv("AZURE_OAI_MODEL_GPT_3")
deployment_name_ada = os.getenv("AZURE_OAI_MODEL_ADA")

os.environ["OPENAI_API_TYPE"] = openai.api_type
os.environ["OPENAI_API_BASE"] = openai.api_base
os.environ["OPENAI_API_KEY"] = openai.api_key
os.environ["OPENAI_API_VERSION"] = openai.api_version

In [15]:
embeddings = OpenAIEmbeddings(
    model= deployment_name_ada, 
    deployment= deployment_name_ada,
    chunk_size=1
    )

In [16]:
# Load from existing index
redis = Redis.from_existing_index(
    embeddings, redis_url="redis://localhost:6379", index_name="dummy-file"
)

In [17]:
question = "What are the medical claims?"
docs = redis.similarity_search_limit_score(question,k=3)
len(docs)

3

In [18]:
llm = ChatOpenAI(model_name=llm_name, temperature=0, engine=deployment_name_gpt)
llm.predict("Hello world!")

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


'Hello there! How can I assist you today?'

In [19]:
# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template,)

In [20]:
# Run chain
from langchain.chains import RetrievalQA
question = "Is there a coverage on claims?"
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=redis.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})


result = qa_chain({"query": question})
result["result"]

'Yes, claims made on-air or in marketing literature that accompanies products will be regulated by the Advertising Standards Authority (ASA), and any direct claim or implication that features on-air will fall within the ASA remit. Most other claims will be regulated by Trading Standards. Thanks for asking!'

In [21]:
#Create a converstaion history
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

In [22]:
# converation retrieval chain
retriever=redis.as_retriever()
qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory
)

In [23]:
question = "Is there a coverage on claims?"
result = qa_chain({"query": question})
result["result"]

'Yes, claims made on-air or in marketing literature that accompanies products will be regulated by the Advertising Standards Authority (ASA), and any direct claim or implication that features on-air will fall within the ASA remit. Most other claims will be regulated by Trading Standards. Thanks for asking!'

In [24]:
question = "Summerize the latest question reponse?"
result = qa_chain({"query": question})
result["result"]

'Sorry, there is no question provided to summarize the latest response.'